# Import libraries

In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from dataset import NCFDataset
from recom_ncf import NCFRecommender
from evaluation import Evaluation
from helpers.index_manager import IndexManager
from helpers.splitter import Splitter
import ast
import numpy as np

# Loading data

In [2]:
metadata_features = ['publishers']
# metadata_features = ['pub_encoded', 'dev_encoded', 'tag_encoded', 'lang_encoded', 'gen_encoded', 'cat_encoded', 'mm_total_recommendation', 'z_total_recommendation', 'log_total_recommendation', 'mm_price', 'z_price', 'log_price', 'mm_released_date', 'z_released_date', 'log_released_date']

In [3]:
df = pd.read_csv('../data/interaction-metadata.csv')[['user_id', 'item_id', 'rating_imp', 'timestamp'] + metadata_features]
df.head()

,user_id,item_id,rating_imp,timestamp,publishers
0,76561197960432447,10,1,1738278781,[0]
1,76561198071230926,10,1,1736206418,[0]
2,76561198206216352,10,1,1738041574,[0]
3,76561198110801124,10,1,1738015332,[0]
4,76561199813732773,10,1,1737853720,[0]


In [4]:
df['publishers'] = df['publishers'].apply(ast.literal_eval)
# df['developers'] = df['developers'].apply(ast.literal_eval)
# df['tags'] = df['tags'].apply(ast.literal_eval)
# df['supported_languages'] = df['supported_languages'].apply(ast.literal_eval)
# df['genres'] = df['genres'].apply(ast.literal_eval)
# df['categories'] = df['categories'].apply(ast.literal_eval)

# Indexing data

In [5]:
index_manager = IndexManager()
index_manager.fit(df_interaction=df)
index_manager.transform_interactions(df, inplace=True)

Indexed 832660 users and 68396 items
User index range: 0-832659
Item index range: 0-68395


,user_id,item_id,rating_imp,timestamp,publishers
0,0,0,1,1738278781,[0]
1,1,0,1,1736206418,[0]
2,2,0,1,1738041574,[0]
3,3,0,1,1738015332,[0]
4,4,0,1,1737853720,[0]
...,...,...,...,...,...
1149950,15755,24642,1,1663545142,[36058]
1149951,150280,24642,1,1689826252,[36058]
1149952,1336,24643,1,1595699739,[1]
1149953,11520,24643,0,1706470035,[1]


In [6]:
df_features = df[['item_id'] + metadata_features].drop_duplicates(subset='item_id')

# Creating datasets

In [7]:
splitter = Splitter(df)
df_train, df_val, df_test = splitter.leave_k_out_split()

Splitting data with leave-2-out strategy (1 for validation, 1 for testing)
Total users: 832660
Interactions per user: min=1, max=1034, avg=1.4
Note: 703199 users have fewer than 2 interactions.
These users will be placed entirely in the training set.
Split complete: 1149955 total interactions
Train set: 891033 interactions (77.5%)
Validation set: 129461 interactions (11.3%)
Test set: 129461 interactions (11.3%)


In [8]:
train_dataset = NCFDataset(df_train)
val_dataset = NCFDataset(df_val)

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=2**16, num_workers=4, persistent_workers=True, prefetch_factor=2, pin_memory=True, shuffle=True)
eval_dataloader = DataLoader(val_dataset, batch_size=2**16, num_workers=4, persistent_workers=True, prefetch_factor=2, pin_memory=True, shuffle=False)

# Training model

In [10]:
mlp_additional_features = {}
for feature in metadata_features:
    sample = df.iloc[0][feature]
    if isinstance(sample, list):
        num_unique_values = df[feature].explode().nunique()
        mlp_additional_features[feature] = (num_unique_values, 8)
    else:
        mlp_additional_features[feature] = (1, 8)

In [11]:
params = {'factors': 8, 'mlp_user_item_dim': 128, 'learning_rate': 0.01, 'epochs': 5, 'optimizer': 'adagrad', 'dropout': 0.0, 'weight_decay': 0.0001, 'loss_fn': 'mse'}

unique_users = index_manager.get_indexed_users()
unique_items = index_manager.get_indexed_items()

model = NCFRecommender(unique_users, unique_items, mlp_additional_features=mlp_additional_features, **params)
model.fit(train_dataloader, eval_dataloader, df_features=df_features)

Epoch 1/5


ValueError: too many values to unpack (expected 3)

# Evaluating model

In [11]:
evaluator = Evaluation(recommender=model, test_data=df_test)
metrics = evaluator.evaluate(user_batch_size=512, item_batch_size=2048)

Creating ground truth sets...
Generating predictions...
Processing predictions for 129461 users and 68396 items
Processing 1 of 129461 users... (0.00%)
Time taken to create additional features: 0.7970438003540039 seconds
Time taken to create additional features: 0.7702267169952393 seconds
Time taken to create additional features: 0.7444725036621094 seconds
Time taken to create additional features: 0.7579104900360107 seconds
Time taken to create additional features: 0.7628574371337891 seconds
Time taken to create additional features: 0.7580218315124512 seconds
Time taken to create additional features: 0.7589056491851807 seconds
Time taken to create additional features: 0.750079870223999 seconds
Time taken to create additional features: 0.7772068977355957 seconds
Time taken to create additional features: 0.7717013359069824 seconds
Time taken to create additional features: 0.7599804401397705 seconds
Time taken to create additional features: 0.7612223625183105 seconds
Time taken to create 

KeyboardInterrupt: 

In [13]:
metrics

{'Hit Ratio@10': 0.03728536006982798,
 'NDCG@10': 0.018122169155170138,
 'Recall@10': 0.03728536006982798}